In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

c:\Users\Win 11\Desktop\RAG Pipeline\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model=SentenceTransformer('all-MiniLM-L6-v2')
## Sample text
text="""
LangChain is a framework for building applications with LLMs.
Langchain provides modular abstractions to combine LLMs with tools like OpenAI and Pinecone.
You can create chains, agents, memory, and retrievers.
The Eiffel Tower is located in Paris.
France is a popular tourist destination.
"""
## Step 1 : Split into sentences
sentences=[s.strip() for s in text.split("\n") if s.strip()]

### Step 2: Embed each sentence
embeddings=model.encode(sentences)

# Step 3: Initialize parameters
threshold = 0.7  # control chunk tightness
chunks = []
current_chunk=[sentences[0]]

## Step 4: Semantic grouping based on threshold

for i in range(1, len(sentences)):
    sim = cosine_similarity(
        [embeddings[i - 1]],
        [embeddings[i]]
    )[0][0]

    if sim>=threshold:
        current_chunk.append(sentences[i])
    else:
        chunks.append(" ".join(current_chunk))
        current_chunk=[sentences[i]]
# Append the last chunk
chunks.append(" ".join(current_chunk))

# Output the chunks
print("Semantic Chunks:")
for idx, chunk in enumerate(chunks):
    print(f"Chunk {idx+1}:{chunk}")

Semantic Chunks:
Chunk 1:LangChain is a framework for building applications with LLMs. Langchain provides modular abstractions to combine LLMs with tools like OpenAI and Pinecone.
Chunk 2:You can create chains, agents, memory, and retrievers.
Chunk 3:The Eiffel Tower is located in Paris.
Chunk 4:France is a popular tourist destination.


In [4]:
### RAG Pipeline Modular Coding

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import init_chat_model
from langchain.schema.runnable import RunnableLambda, RunnableMap
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [5]:
### Custom Semantic Chunker With Threshold

class ThresholdSemanticChunker:
    def __init__(self,model_name="all-MiniLM-L6-v2",threshold=0.7):
        self.model=SentenceTransformer(model_name)
        self.threshold=threshold

    def split(self, text: str):
        sentences = [s.strip() for s in text.split('.') if s.strip()]
        embeddings = self.model.encode(sentences)
        chunks = []
        current_chunk = [sentences[0]]

        for i in range(1, len(sentences)):
            sim = cosine_similarity([embeddings[i - 1]], [embeddings[i]])[0][0]
            if sim >= self.threshold:
                current_chunk.append(sentences[i])
            else:
                chunks.append(". ".join(current_chunk) + ".")
                current_chunk = [sentences[i]]

        chunks.append(". ".join(current_chunk) + ".")
        return chunks

    def split_documents(self,docs):
        result=[]
        for doc in docs:
            for chunk in self.split(doc.page_content):
                result.append(Document(page_content=chunk, metadata=doc.metadata))

        return result

In [6]:
# Sample text
sample_text = """
    "LangChain is a framework for building applications with LLMs."
    "Langchain provides modular abstractions to combine LLMs with tools like OpenAI and Pinecone."
    "You can create chains, agents, memory, and retrievers."
    "The Eiffel Tower is located in Paris."
    "France is a popular tourist destination."
    """
doc = Document(page_content=sample_text)
doc

Document(metadata={}, page_content='\n    "LangChain is a framework for building applications with LLMs."\n    "Langchain provides modular abstractions to combine LLMs with tools like OpenAI and Pinecone."\n    "You can create chains, agents, memory, and retrievers."\n    "The Eiffel Tower is located in Paris."\n    "France is a popular tourist destination."\n    ')

In [7]:
### Chunking
chunker=ThresholdSemanticChunker(threshold=0.7)
chunks=chunker.split_documents([doc])
chunks

[Document(metadata={}, page_content='"LangChain is a framework for building applications with LLMs. "\n    "Langchain provides modular abstractions to combine LLMs with tools like OpenAI and Pinecone.'),
 Document(metadata={}, page_content='"\n    "You can create chains, agents, memory, and retrievers.'),
 Document(metadata={}, page_content='"\n    "The Eiffel Tower is located in Paris.'),
 Document(metadata={}, page_content='"\n    "France is a popular tourist destination.'),
 Document(metadata={}, page_content='".')]

In [8]:
### VectorStore
import os
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
embedding=OpenAIEmbeddings()
vectorstore=FAISS.from_documents(chunks,embedding)
retriever=vectorstore.as_retriever()


In [10]:
## Prompt Template
template = """Answer the question based on the following context:
    {context}
    Question: {question}
    """
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based on the following context:\n    {context}\n    Question: {question}\n    ')

In [11]:
from langchain.chat_models.base import init_chat_model
llm=init_chat_model("openai:gpt-3.5-turbo")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002469134F590>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002469134F620>, root_client=<openai.OpenAI object at 0x0000024691263E60>, root_async_client=<openai.AsyncOpenAI object at 0x000002469134E240>, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [13]:
### LCEL Chain With retrieval
rag_chain=(
        RunnableMap(
            {
            "context": lambda x: retriever.invoke(x["question"]),
            "question": lambda x: x["question"],
            }
        )
        | prompt
        | llm
        | StrOutputParser()
    )
query = {"question": "What is LangChain used for?"}
result = rag_chain.invoke(query)

print(result)

LangChain is a framework used for building applications with Large Language Models (LLMs). It provides modular abstractions to combine LLMs with tools like OpenAI and Pinecone.


In [15]:
from langchain_openai import OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain.document_loaders import TextLoader

In [16]:
## Load the documents
loader=TextLoader("langchain_intro.txt")
docs=loader.load()

## Initialize embedding model
embedding=OpenAIEmbeddings()

## Create the semantic chunker
chunker=SemanticChunker(embedding)

## Split the documents
chunks=chunker.split_documents(docs)

## Result

for i,chunk in enumerate(chunks):
    print(f"\n chunk {i+1}:\n{chunk.page_content}")


 chunk 1:
LangChain is a framework for building applications with LLMs. Langchain provides modular abstractions to combine LLMs with tools like OpenAI and Pinecone.

 chunk 2:
You can create chains, agents, memory, and retrievers. The Eiffel Tower is located in Paris. France is a popular tourist destination.
